## Web Scrapping For Prediction

In [1]:
import warnings
import requests
from sklearn import metrics
import pandas as pd
from bs4 import BeautifulSoup
warnings.filterwarnings('ignore')
import numpy as np

In [2]:
total_form = []
total_links = []
data = requests.get('https://www.thegreyhoundrecorder.com.au/form-guides/')
soup = BeautifulSoup(data.text,'html.parser')
linkss = soup.find_all('table',{'class':'fields table-striped'})
for i in linkss:
    links = soup.find_all('a')
    for l in links:
        link = l.get('href')
        if 'fields' in link:
            link = 'https://www.thegreyhoundrecorder.com.au/'+link
            total_form.append(link)
            
total_form = list(dict.fromkeys(total_form))

In [3]:
import datetime
a=datetime.date.today() + datetime.timedelta(days=1)
a=a.strftime("%d/%m/%Y")
DateT=str(a).replace('-','/')[:-2]
DateT

'08/10/20'

In [4]:
NextDayLinks=[]
for i in total_form:
  l = requests.get(i)
  soup = BeautifulSoup(l.text,'html.parser')
  title=soup.find('title')
  title=title.text
  Date=title.split('- ')[1]
  
  if Date==DateT:
    NextDayLinks.append(i)
    print(i)

https://www.thegreyhoundrecorder.com.au//form-guides/addington/fields/72223
https://www.thegreyhoundrecorder.com.au//form-guides/albion-park/fields/72232
https://www.thegreyhoundrecorder.com.au//form-guides/cambridge/fields/72220
https://www.thegreyhoundrecorder.com.au//form-guides/casino/fields/72226
https://www.thegreyhoundrecorder.com.au//form-guides/dapto/fields/72225
https://www.thegreyhoundrecorder.com.au//form-guides/gawler/fields/72208
https://www.thegreyhoundrecorder.com.au//form-guides/geelong/fields/72214
https://www.thegreyhoundrecorder.com.au//form-guides/hobart/fields/72209
https://www.thegreyhoundrecorder.com.au//form-guides/mandurah/fields/72215
https://www.thegreyhoundrecorder.com.au//form-guides/mount-gambier/fields/72238
https://www.thegreyhoundrecorder.com.au//form-guides/sale/fields/72219
https://www.thegreyhoundrecorder.com.au//form-guides/sandown-park/fields/72224
https://www.thegreyhoundrecorder.com.au//form-guides/shepparton/fields/72218
https://www.thegreyhoun

In [5]:
df1st=pd.DataFrame(columns=['Grade','Box','Dist','Name','Track','Race'])
count2loop=1
for k in range(len(NextDayLinks)):
  l = requests.get(NextDayLinks[k])
  soup = BeautifulSoup(l.text,'html.parser')
  Content=soup.find_all('div',{'class':'fieldsSingleRace'})

  for j in range(len(soup.findAll('table',{'class':'fieldsTable'}))):
    Grade=[]
    Box=[]
    Distance=[]
    DogName=[]
    TrackFull=[]
    count2loopList=[]
    Grade1=soup.findAll('table',{'class':'fieldsTable'})[j]
    count=0
    for i in range(len(Content[j].findAll('span',{'class':'dogName'}))):
      if count<7:
        Grade2=Grade1.find_all('tr',{'class':'selection-row'})[count]
        try:
          Grade.append(Grade2.findAll('td')[3].text)
          Box.append(int(Content[j].findAll('span',{'class':'dogCarpet'})[count].text))
          Distance.append(Content[j].findAll('div',{'class':'fieldsTopHeader'})[0].div.text.split('(')[-1].split('m)')[0])
          L = Content[j].findAll('span',{'class':'dogName'})[count].text
          L=L.title()
          L=L.replace(' ' , '-')
          DogName.append(L)
          TrackFull.append(soup.find('title').text.split('- ')[0].split(' Greyhound ')[0])
          count+=1
          count2loopList.append('Race ' + str(count2loop))
        except Exception as e:
          count+=1
    Bt=np.array([Grade,Box,Distance,DogName,TrackFull,count2loopList]).T
    df1st=df1st.append(pd.DataFrame(Bt, columns=list(df1st)), ignore_index=True)
    count2loop+=1
  count2loop=0

In [6]:
df1st=df1st.drop(df1st[df1st.Grade == ""].index)

In [7]:
TrackShort=pd.read_csv("/content/drive/My Drive/DOG RACE/de_para-2.csv")

In [8]:
TrackShort=TrackShort.dropna()

In [9]:

SName=TrackShort['Short'].values.tolist()
FName=TrackShort['Full'].values.tolist()

In [10]:
TrackCompareDic = {FName[i]: SName[i] for i in range(len(FName))} 

In [11]:
df1st=df1st.replace({"Track": TrackCompareDic})

In [12]:
df1st

,Grade,Box,Dist,Name,Track,Race
0,3,1,295,Our-Cook,CCH,Race 1
1,3,2,295,No-Kissy-Missy,CCH,Race 1
2,3,3,295,Mitcham-Boult,CCH,Race 1
3,3,4,295,Goldstar-Flora,CCH,Race 1
4,3,5,295,Taieri-Terra,CCH,Race 1
...,...,...,...,...,...,...
1061,5,3,390,Notorious,WBL,Race 11
1062,5,4,390,Ragin'-Soldier,WBL,Race 11
1063,5,5,390,Vin-Joel,WBL,Race 11
1064,5,6,390,Rampant-Teresa,WBL,Race 11


In [13]:
remdup = pd.read_csv("/content/drive/My Drive/DOG RACE/remdup_data (1).csv")

In [14]:
remdup['Date']=remdup['Date'].replace('[/-]','-',regex=True)
remdup['Date']=pd.to_datetime(remdup['Date'])
remdup.head(5)

,Unnamed: 0,Date,Dog,Track,Fin,Box,Dist,Grade,Time,Win T
0,0,2020-08-21,Give-Me-Ruby's,APK,6th,1.0,388.0,M,23.56,22.74
1,1,2020-07-31,Give-Me-Ruby's,GAW,6th,1.0,400.0,M,24.26,23.54
2,2,2020-08-18,Bacardi-Blaze,GAW,4th,7.0,400.0,M,23.76,22.92
3,3,2020-08-11,Bacardi-Blaze,GAW,2nd,4.0,400.0,M,23.80,23.51
4,4,2020-07-27,Bacardi-Blaze,APK,5th,4.0,388.0,M,23.25,22.06


In [15]:
remdup = remdup.drop_duplicates()
remdup = remdup.reset_index(drop = True)

In [16]:
remdup = remdup[["Unnamed: 0","Date","Dog","Track","Fin","Box","Dist","Grade","Time","Win T"]]
remdup['Date'] = pd.to_datetime(remdup.Date)
remdup["Date"] = remdup["Date"].astype(str)
remdup["Win T"] = remdup["Win T"].astype(str)
remdup["Dist"] = remdup["Dist"].astype(str)
remdup["concatenated_values"] = remdup["Date"]+remdup["Track"]+remdup["Win T"]+remdup["Dist"]

remdup = remdup.drop(columns=["Unnamed: 0"])

remdup = remdup.drop_duplicates()

remdup

,Date,Dog,Track,Fin,Box,Dist,Grade,Time,Win T,concatenated_values
0,2020-08-21,Give-Me-Ruby's,APK,6th,1.0,388.0,M,23.56,22.74,2020-08-21APK22.74388.0
1,2020-07-31,Give-Me-Ruby's,GAW,6th,1.0,400.0,M,24.26,23.54,2020-07-31GAW23.54400.0
2,2020-08-18,Bacardi-Blaze,GAW,4th,7.0,400.0,M,23.76,22.92,2020-08-18GAW22.92400.0
3,2020-08-11,Bacardi-Blaze,GAW,2nd,4.0,400.0,M,23.80,23.51,2020-08-11GAW23.51400.0
4,2020-07-27,Bacardi-Blaze,APK,5th,4.0,388.0,M,23.25,22.06,2020-07-27APK22.06388.0
...,...,...,...,...,...,...,...,...,...,...
357271,2020-09-29,Little-Miss-Asha,DEV,7th,3.0,452.0,5,26.49,25.6,2020-09-29DEV25.6452.0
357272,2020-09-28,Winklee-Wizard,LCN,4th,8.0,278.0,5,16.54,16.32,2020-09-28LCN16.32278.0
357273,2020-09-28,Gee-Whiz-Dolly,LCN,3rd,8.0,278.0,5,16.70,16.36,2020-09-28LCN16.36278.0
357274,2020-09-29,Minish-Woods,DEV,5th,7.0,452.0,4,26.25,25.75,2020-09-29DEV25.75452.0


In [17]:
count = remdup['concatenated_values'].value_counts() == 8
count = pd.DataFrame(count)
count = count.reset_index()
cols = count.columns.values
cols[0] = "concatenated_values"
cols[1] = "value"
count.columns = cols
count["value"] = count["value"].astype(str)
updated_count = count.loc[count['value'] == "True"]
concatenated_values = updated_count.concatenated_values.tolist()
df = remdup.loc[remdup['concatenated_values'].isin(concatenated_values)]

remdup.shape


(357276, 10)

In [18]:
df["cluster"] = 0
uniqe = df.concatenated_values.unique()
i = 0
while i < len(uniqe):
    df.loc[df['concatenated_values'] == uniqe[i], 'cluster'] = i+1
    i = i+1
final_data = df.sort_values('cluster')
final_data = final_data.reset_index(drop = True)
final_data = final_data[["Date","Dog","Track","Fin","Box","Dist","Grade","Time","Win T","cluster"]]
final_data.head(200)

,Date,Dog,Track,Fin,Box,Dist,Grade,Time,Win T,cluster
0,2020-08-16,Andy's-Handy,MBR,5th,6.0,395.0,M,23.11,22.57,1
1,2020-08-16,Shadow's-Star,MBR,7th,1.0,395.0,M,23.22,22.57,1
2,2020-08-16,Cawbourne-Jumpy,MBR,8th,4.0,395.0,M,23.28,22.57,1
3,2020-08-16,Hali's-Girl,MBR,3rd,2.0,395.0,M,22.75,22.57,1
4,2020-08-16,Sidney's-Star,MBR,6th,7.0,395.0,M,23.11,22.57,1
...,...,...,...,...,...,...,...,...,...,...
195,2020-06-24,Zoometrical,APK,7th,6.0,388.0,SNP,23.06,22.5,25
196,2020-06-24,Emerley-Jose,APK,3rd,1.0,388.0,SNP,22.74,22.5,25
197,2020-06-24,Burning-Glow,APK,6th,7.0,388.0,SNP,22.89,22.5,25
198,2020-06-24,Filthy-Phantom,APK,5th,8.0,388.0,SNP,22.89,22.5,25


In [19]:
print("Maximum value of the Cluster: ",max(final_data.cluster))

Maximum value of the Cluster:  4298


In [20]:
final_data

,Date,Dog,Track,Fin,Box,Dist,Grade,Time,Win T,cluster
0,2020-08-16,Andy's-Handy,MBR,5th,6.0,395.0,M,23.11,22.57,1
1,2020-08-16,Shadow's-Star,MBR,7th,1.0,395.0,M,23.22,22.57,1
2,2020-08-16,Cawbourne-Jumpy,MBR,8th,4.0,395.0,M,23.28,22.57,1
3,2020-08-16,Hali's-Girl,MBR,3rd,2.0,395.0,M,22.75,22.57,1
4,2020-08-16,Sidney's-Star,MBR,6th,7.0,395.0,M,23.11,22.57,1
...,...,...,...,...,...,...,...,...,...,...
34379,2020-09-27,Pacino,DAR,3rd,2.0,312.0,4,18.82,18.48,4298
34380,2020-09-27,Spirit-Bar,DAR,5th,7.0,312.0,4,18.96,18.48,4298
34381,2020-09-27,Kabo's-Winlock,DAR,7th,4.0,312.0,4,19.20,18.48,4298
34382,2020-09-27,Sunset-Inferno,DAR,2nd,1.0,312.0,4,18.65,18.48,4298


In [21]:
dictDog={}
for i in range(remdup['Dog'].unique().shape[0]):
  dictDog.update({remdup['Dog'].unique()[i]:i})

In [22]:
dictDog

{"Give-Me-Ruby's": 0,
 'Bacardi-Blaze': 1,
 'Galaxyia-Moon': 2,
 'Danger-Dan': 3,
 'Beers-With-Geoff': 4,
 'Go-Sister-Go': 5,
 'Hot-Rocks': 6,
 "Andy's-Handy": 7,
 'For-The-Thrill': 8,
 'Little-Herbie': 9,
 'Deedy': 10,
 'Straight-Blaze': 11,
 'Sallynoggin': 12,
 'Allegra': 13,
 'Gypsy-By-Design': 14,
 'Mobius-Loop': 15,
 "Bilman's-Stunner": 16,
 'Kick-Up': 17,
 'Starlight-Meg': 18,
 'Thomas-Shelby': 19,
 'Go-Fashion-Girl': 20,
 'Vibrant-Lee': 21,
 'Cordoza': 22,
 'Crazy-Jake': 23,
 'Dinkydi-Sheila': 24,
 "Elsie's-Choice": 25,
 'They-Call-Me': 26,
 'Hurricane-Evy': 27,
 'Sydney-Cove': 28,
 'Wood-Veneer': 29,
 "Tilly's-Charm": 30,
 'My-Little-Viper': 31,
 'Zipping-Arden': 32,
 'Graceful-Gazelle': 33,
 'Sneaky-Option': 34,
 'Gangalla-Ninja': 35,
 'Rock-It-Fabio': 36,
 'Power-Stone': 37,
 "Charlie's-Chance": 38,
 'Westgate-Girl': 39,
 'Loud-Missile': 40,
 'Absolute-Dream': 41,
 'Just-Kenny': 42,
 'Release-The-Fury': 43,
 'Huey-Lee': 44,
 'Ripple-Ryder': 45,
 'Peregrine-Flight': 46,
 'King

In [23]:
remdup.Dog=[dictDog[item]for item in remdup.Dog]
final_data.Dog=[dictDog[item]for item in final_data.Dog]
count1=1
count2=1
for item in df1st.Name:
  if item not in dictDog.keys():
    df1st = df1st.drop(df1st[df1st.Name == item].index)
    count1+=1
  else:
    count2+=1
df1st.Name=[dictDog[item]for item in df1st.Name]
print('not',count1,'yes',count2)

not 157 yes 905


In [26]:
df1st

,Grade,Box,Dist,Name,Track,Race
0,3,1,295,6663,CCH,Race 1
1,3,2,295,6695,CCH,Race 1
2,3,3,295,6705,CCH,Race 1
3,3,4,295,6692,CCH,Race 1
4,3,5,295,6666,CCH,Race 1
...,...,...,...,...,...,...
1061,5,3,390,8922,WBL,Race 11
1062,5,4,390,3708,WBL,Race 11
1063,5,5,390,5901,WBL,Race 11
1064,5,6,390,5902,WBL,Race 11


In [27]:
dictTrack={}
for i in range(remdup['Track'].unique().shape[0]):
  dictTrack.update({remdup['Track'].unique()[i]:i})

In [28]:
remdup.Track=[dictTrack[item]for item in remdup.Track]
final_data.Track=[dictTrack[item]for item in final_data.Track]
df1st.Track=[dictTrack[item]for item in df1st.Track]

In [29]:
dictGrade={}
for i in range(remdup['Grade'].unique().shape[0]):
  dictGrade.update({remdup['Grade'].unique()[i]:i})

In [30]:
remdup.Grade=[dictGrade[item]for item in remdup.Grade]
final_data.Grade=[dictGrade[item]for item in final_data.Grade]
df1st.Grade=[dictGrade[item]for item in df1st.Grade]

In [31]:

dictFin={}
for i in range(remdup['Fin'].unique().shape[0]):
  dictFin.update({remdup['Fin'].unique()[i]:i})

In [32]:
remdup.Fin=[dictFin[item]for item in remdup.Fin]
final_data.Fin=[dictFin[item]for item in final_data.Fin]

In [33]:
TrackList=list(dictTrack.keys())

In [34]:
remdup

,Date,Dog,Track,Fin,Box,Dist,Grade,Time,Win T,concatenated_values
0,2020-08-21,0,0,0,1.0,388.0,0,23.56,22.74,2020-08-21APK22.74388.0
1,2020-07-31,0,1,0,1.0,400.0,0,24.26,23.54,2020-07-31GAW23.54400.0
2,2020-08-18,1,1,1,7.0,400.0,0,23.76,22.92,2020-08-18GAW22.92400.0
3,2020-08-11,1,1,2,4.0,400.0,0,23.80,23.51,2020-08-11GAW23.51400.0
4,2020-07-27,1,0,3,4.0,388.0,0,23.25,22.06,2020-07-27APK22.06388.0
...,...,...,...,...,...,...,...,...,...,...
357271,2020-09-29,7579,44,8,3.0,452.0,10,26.49,25.6,2020-09-29DEV25.6452.0
357272,2020-09-28,2925,45,1,8.0,278.0,10,16.54,16.32,2020-09-28LCN16.32278.0
357273,2020-09-28,2971,45,4,8.0,278.0,10,16.70,16.36,2020-09-28LCN16.36278.0
357274,2020-09-29,4595,44,3,7.0,452.0,35,26.25,25.75,2020-09-29DEV25.75452.0


In [35]:
pred_x=df1st[['Name','Track','Box','Dist','Grade']]
pred_x = pred_x.drop(pred_x[pred_x.Name == 'Strike-It-Lucky'].index)
pred_x.head(20)

,Name,Track,Box,Dist,Grade
0,6663,71,1,295,66
1,6695,71,2,295,66
2,6705,71,3,295,66
3,6692,71,4,295,66
4,6666,71,5,295,66
5,11140,71,6,295,66
6,5645,71,7,295,66
8,6706,71,2,295,35
9,5667,71,3,295,35
10,9925,71,4,295,35


In [36]:
train_x=remdup[['Dog','Track','Box','Dist','Grade']]

In [37]:
train_y=remdup['Time']

In [38]:
test_x=final_data[['Dog','Track','Box','Dist','Grade']]

In [39]:
test_y=final_data['Time']

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from sklearn.model_selection import train_test_split
import seaborn as sn
import time
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report 
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LinearRegression 

In [41]:
ModelName=[]
TestAccuracy=[]
TrainAccuracy=[]
PredictedResult=[]
# ConfusionMatrix=[]
# Probes=[]

In [42]:
def Train(name,model):
    ModelName.append(name)
    model.fit(train_x, train_y)
    y_predicted = model.predict(pred_x)
    PredictedResult.append(list(y_predicted))
    train_Acc = model.score(train_x, train_y)
    print('train accuracy for ',name,'=',train_Acc)
    TrainAccuracy.append(train_Acc)
    # test_Acc = model.score(test_y, y_predicted)
    test_Acc=model.score(test_x,test_y)
    TestAccuracy.append(test_Acc)
    print('test accuracy for ',name,'=',test_Acc)
    print('\n')
    # ConfusionMatrix.append(confusion_matrix(test_y,y_predicted))
    # Probes.append(model.predict_proba(test_x))

In [43]:
models=[LassoCV(),RidgeCV(),DecisionTreeRegressor(),RandomForestRegressor(n_estimators = 100, random_state = 0),ElasticNetCV(),LinearRegression()]
names=['LASSO Regression','Ridge Regression','Decision Tree','Random Forest','ElasticNet Regression','linear regression']

In [44]:
t1 = time.perf_counter()
for i in range(len(names)):
    Train(names[i],models[i])
t2 = time.perf_counter()
print('\n')
print(f'Finished in {t2-t1} seconds')

train accuracy for  LASSO Regression = 0.8308730106715658
test accuracy for  LASSO Regression = 0.8229750926907875


train accuracy for  Ridge Regression = 0.830911082030654
test accuracy for  Ridge Regression = 0.8231120283376844


train accuracy for  Decision Tree = 0.9794193020968727
test accuracy for  Decision Tree = 0.9817503890550722


train accuracy for  Random Forest = 0.9573931545086608
test accuracy for  Random Forest = 0.9573117204400299


train accuracy for  ElasticNet Regression = 0.8308723029838423
test accuracy for  ElasticNet Regression = 0.8229750101245026


train accuracy for  linear regression = 0.8309110820728934
test accuracy for  linear regression = 0.8231120558731211




Finished in 133.88321297799985 seconds


In [45]:
predictAvg=[]
for j in range(len(PredictedResult[0])):
  avg=0
  for i in range(len(PredictedResult)):
    avg+=PredictedResult[i][j]
  predictAvg.append(avg/len(PredictedResult))

In [46]:
df1st['Predicted Time']=predictAvg

In [47]:
dictDog2 = {v : k for k, v in dictDog.items()}
df1st.Name=[dictDog2[item]for item in df1st.Name]

In [48]:
dictTrack2 = {v : k for k, v in dictTrack.items()}
df1st.Track=[dictTrack2[item]for item in df1st.Track]

In [49]:
dictGrade2 = {v : k for k, v in dictGrade.items()}
df1st.Grade=[dictGrade2[item]for item in df1st.Grade]

In [50]:
df1st

,Grade,Box,Dist,Name,Track,Race,Predicted Time
0,3,1,295,Our-Cook,CCH,Race 1,17.414009
1,3,2,295,No-Kissy-Missy,CCH,Race 1,17.443502
2,3,3,295,Mitcham-Boult,CCH,Race 1,17.466849
3,3,4,295,Goldstar-Flora,CCH,Race 1,17.430325
4,3,5,295,Taieri-Terra,CCH,Race 1,17.472728
...,...,...,...,...,...,...,...
1061,5,3,390,Notorious,WBL,Race 11,22.544119
1062,5,4,390,Ragin'-Soldier,WBL,Race 11,22.800424
1063,5,5,390,Vin-Joel,WBL,Race 11,22.558372
1064,5,6,390,Rampant-Teresa,WBL,Race 11,22.628693
